In [ ]:
import psfmodels as psfm
import numpy as np
import tifffile
from pathlib import Path

# ---- Acquisition parameters ----
wavelength = 0.525          # µm (eGFP)
nx = 65                     # must be odd
nz = 33                     # must be odd
dxy = 0.347                 # µm / pixel
dz = 2.0                    # µm step in Z
pz = 0.0                    # point source in focus

# ---- Microscope/detection parameters ----
params = {
    "NA": 1.1,         # Objective NA
    "ni": 1.333,       # Immersion RI (water)
    "ns": 1.45,        # Sample RI
}

# ---- Simulate vectorial detection PSF ----
psf_det = psfm.vectorial_psf_centered(
    nz=nz, dz=dz,
    nx=nx, dxy=dxy,
    wvl=wavelength,
    pz=pz,
    params=params,
)

# ---- Model light sheet as Gaussian excitation in Z ----
fwhm_exc_z = 3.0  # µm (light sheet axial thickness)
sigma_exc_z = fwhm_exc_z / 2.355

z_vals = np.linspace(-(nz // 2) * dz, (nz // 2) * dz, nz)
exc_profile = np.exp(-(z_vals ** 2) / (2 * sigma_exc_z ** 2))
exc_profile = exc_profile[:, None, None]  # shape (nz, 1, 1)

# ---- Combine illumination × detection ----
psf_eff = psf_det * exc_profile
psf_eff /= psf_eff.sum()  # normalize to sum=1

# ---- Save the effective PSF ----
out_dir = Path("/media/kenzo/La Vanoise/JSDX83_16bit_tiff")
out_dir.mkdir(exist_ok=True)
out_file = out_dir / "psf_488nm_vectorial.tif"

tifffile.imwrite(out_file, psf_eff.astype(np.float32))
print(" Effective PSF saved:", out_file.name, "| shape:", psf_eff.shape)